# Example on how to add data to the dataframe (database)

Here we need to add new weather columns to the dataframe. Specifically:

```python
cols = [
    "precipitation",
    "wind_gusts_10m",
    "cloud_cover",
    'shortwave_radiation'
]
```

This data can be obtained from the same API so the data collection is strigtforward. 
First, we update `variables_standard` in Openmeteo class to add new quantities. This will assure that they are downloaded for all future data updates. However they also need to be added to the dataframe itself. This can be accomplised as follows. 

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
df_original = pd.read_parquet('../database/prev_latest.parquet')

In [ ]:
df_original.tail()

In [ ]:
start_date = pd.Timestamp(df_original.dropna(how='any',inplace=False).first_valid_index())
today = pd.Timestamp(datetime.today())
end_date = pd.Timestamp(df_original.dropna(how='any',inplace=False).last_valid_index())
from data_modules.collect_data_openmeteo import get_weather_data_from_api_forecast, get_weather_data_from_api, \
    locations, OpenMeteo

df_om_hist = get_weather_data_from_api(start_date, today-timedelta(hours=12), locations)



In [ ]:
df_om_forecast = get_weather_data_from_api_forecast(locations=locations)
if not df_om_forecast.columns.equals(df_om_hist.columns):
    print("! Error. Column mismatch between historical and forecasted weather!")
df_om = pd.concat([df_om_hist, df_om_forecast[df_om_hist.columns]], ignore_index=True)
df_om.drop_duplicates(subset='date', keep='last', inplace=True)
# df_om = process_weather_quantities(df_om, locations)
df_om.set_index('date',inplace=True)

In [ ]:
df_om

In [ ]:
# df_om.to_parquet('../database'+'db_openweather.parquet',engine='pyarrow')
    

In [ ]:
from data_modules.collect_data_openmeteo import OpenMeteo
for var in OpenMeteo.variables_standard:
    for col in df_original.columns.to_list():
        if str(col).__contains__(var):
            df_original.drop(col, axis=1, inplace=True)

In [ ]:
df_original

In [ ]:
df_original = df_original.join(df_om)

In [ ]:
df_original

# Add SMARD columns

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
df_original = pd.read_parquet('../database/latest.parquet')

In [ ]:
df_original

In [ ]:
start_date = pd.Timestamp(df_original.dropna(how='any',inplace=False).first_valid_index())
today = pd.Timestamp(datetime.today())
end_date = pd.Timestamp(df_original.dropna(how='any',inplace=False).last_valid_index())

In [ ]:
from data_modules.collect_data_smard import DataEnergySMARD
o_smard = DataEnergySMARD(start_date=start_date, end_date=end_date)
df_smard_flow = o_smard.get_international_flow()
df_smard_flow

In [ ]:
df_smard_flow.set_index('date',inplace=True)

In [ ]:

for col in df_smard_flow.columns.to_list():
    if not col in df_original.columns.to_list():
        print(f"Adding...{col}")
        # merge the new column with the dataframe 
        df_original = df_original.merge(df_smard_flow[[col]], how='left', left_index=True, right_index=True)
df_original.to_parquet('../database/latest.parquet')

In [ ]:
d